# Part-of-Speech (POS) Tagging Project

## Introduction

Part-of-Speech (POS) tagging is the task of assigning a grammatical category (such as noun, verb, adjective, etc.) to each word in a sentence. POS tags provide essential syntactic information that is useful in many natural language processing (NLP) applications, such as:

- Text parsing and syntactic analysis
- Named entity recognition
- Machine translation
- Information extraction

In this project, we use the **Penn Treebank (PTB) corpus**, which is a well-known annotated dataset containing English sentences where each word is labeled with its correct POS tag.

---

## Objective

The main goal of this project is to **predict the POS tags for words in a sentence** using a statistical approach. We aim to:

1. Build a **vocabulary** of words from the training dataset.
2. Handle **unknown words** using morphological rules and special tokens.
3. Construct **transition** and **emission probability matrices** from the training data.
4. Implement the **Viterbi algorithm** to find the most likely sequence of POS tags for a given sentence.
5. Evaluate the performance using **accuracy** on a test set.

---

## Approach

### 1. Data Preparation

- Load the **Penn Treebank** corpus from NLTK.
- Split the corpus into **training** (90%) and **testing** (10%) sets.
- Create a **vocabulary dictionary** mapping each word to a unique integer ID.
- Preprocess the test corpus to replace unknown words with special tokens such as `--unk--`, `--unk_digit--`, or `--unk_noun--` based on simple morphological rules.

### 2. Handling Unknown Words

Unknown words are replaced using rules based on:

- Digits → `--unk_digit--`
- Uppercase letters → `--unk_upper--`
- Punctuation → `--unk_punct--`
- Common suffixes for nouns, verbs, adjectives, and adverbs → respective unknown tokens

This helps the model generalize better to words not seen during training.

### 3. Statistical Model

We use a **Hidden Markov Model (HMM)** approach:

- **Transition probabilities (A)**: Probability of tag_j given tag_i, i.e., \( P(t_j | t_i) \)
- **Emission probabilities (B)**: Probability of word given tag, i.e., \( P(w | t) \)

These are calculated from the training corpus with **additive smoothing** to avoid zero probabilities.

### 4. Viterbi Algorithm

The **Viterbi algorithm** is used to find the most likely sequence of POS tags for a given sequence of words. Steps:

1. **Initialization**: Compute probabilities for the first word using the start state and emission probabilities.
2. **Forward recursion**: For each word, compute the maximum probability of reaching each POS tag considering all possible previous tags.
3. **Backtracking**: Recover the best sequence of POS tags using the stored backpointers.

We implement the Viterbi algorithm using **log probabilities** to prevent numerical underflow caused by multiplying many small probabilities.

### 5. Evaluation

The predicted tags are compared against the ground truth in the test corpus. Accuracy is computed as:

$$
\text{Accuracy} = \frac{\text{Number of correct tags}}{\text{Total number of words}}
$$




---

# Section 1: Data Loading, Vocabulary Building, and Preprocessing

---

## Data Loading

The first step is to get the **Treebank corpus**, which is a collection of English sentences where each word is already labeled with its part-of-speech (POS) tag. We first check if the dataset is available, and if not, we download it. This corpus gives us sentences with correct word-tag pairs, which we will use to train and test our POS tagging model.

We also make sure that the tools needed to split text into sentences and words are ready. This helps us prepare the data in a format suitable for further steps like building vocabulary and training the model.

---

## Vocabulary Building

Once the data is loaded, the next step is to make a **vocabulary**, which is a list of all unique words in the dataset. This vocabulary is important because:

- It helps the model recognize words it has seen before.
- It lets us assign a number to each word so the model can work with them in calculations.
- It helps us deal with words that are not in the training data.

Having a clear and complete vocabulary ensures the model can learn the correct relationship between words and their POS tags.

---

## Handling Unknown Words

In real text, we often see words that are not in our training dataset. These are called **unknown words**. To handle them, we assign special labels based on simple rules. For example, we check if a word has a number, punctuation, capital letters, or certain endings that hint at its type (like nouns or verbs). These special labels allow the model to guess the POS tag for unknown words more accurately.

---

## Preprocessing the Corpus

Before using the data for training, we need to **preprocess** it. Preprocessing does a few things:

1. Turns all sentences into a single sequence of words for easier processing.
2. Replaces unknown words with their special labels from the previous step.
3. Marks the end of each sentence to help the model understand sentence boundaries.

This step makes sure the data is organized in a way the model can use, and prepares it for probabilistic methods like Hidden Markov Models (HMM) and algorithms like Viterbi.

---

## Dataset Splitting

To check how well our model works, we split the dataset into **training** and **test** sets. Most of the data is used for training so the model can learn patterns between words and POS tags. The remaining smaller portion is used for testing, which lets us see how well the model predicts tags on words it has not seen before. This ensures the results reflect the model’s true ability to generalize, not just memorization of the training data.


In [1]:
import pandas as pd
from collections import defaultdict
import math
import numpy as np


In [2]:
import nltk
print("Checking for 'treebank' corpus...")

try:
    nltk.data.find('corpora/treebank')
    print("'treebank' corpus found.")
except LookupError:
    print("'treebank' corpus not found. Downloading...")
    nltk.download('treebank')
    print("'treebank' corpus downloaded successfully.")

from nltk.corpus import treebank

tagged_sentences = treebank.tagged_sents()

print(f"Number of sentences in treebank: {len(tagged_sentences)}")

# Print a few sample sentences
print("\nSample tagged sentences:")
print(tagged_sentences[0:3])


Checking for 'treebank' corpus...
'treebank' corpus found.
Number of sentences in treebank: 3914

Sample tagged sentences:
[[('Pierre', 'NNP'), ('Vinken', 'NNP'), (',', ','), ('61', 'CD'), ('years', 'NNS'), ('old', 'JJ'), (',', ','), ('will', 'MD'), ('join', 'VB'), ('the', 'DT'), ('board', 'NN'), ('as', 'IN'), ('a', 'DT'), ('nonexecutive', 'JJ'), ('director', 'NN'), ('Nov.', 'NNP'), ('29', 'CD'), ('.', '.')], [('Mr.', 'NNP'), ('Vinken', 'NNP'), ('is', 'VBZ'), ('chairman', 'NN'), ('of', 'IN'), ('Elsevier', 'NNP'), ('N.V.', 'NNP'), (',', ','), ('the', 'DT'), ('Dutch', 'NNP'), ('publishing', 'VBG'), ('group', 'NN'), ('.', '.')], [('Rudolph', 'NNP'), ('Agnew', 'NNP'), (',', ','), ('55', 'CD'), ('years', 'NNS'), ('old', 'JJ'), ('and', 'CC'), ('former', 'JJ'), ('chairman', 'NN'), ('of', 'IN'), ('Consolidated', 'NNP'), ('Gold', 'NNP'), ('Fields', 'NNP'), ('PLC', 'NNP'), (',', ','), ('was', 'VBD'), ('named', 'VBN'), ('*-1', '-NONE-'), ('a', 'DT'), ('nonexecutive', 'JJ'), ('director', 'NN'), ('

In [3]:
import nltk
from nltk.corpus import treebank

# 1. Check/download punkt tokenizer
print("Checking for 'punkt' tokenizer...")

try:
    nltk.data.find('tokenizers/punkt')
    print("'punkt' tokenizer found.")
except LookupError:
    print("'punkt' tokenizer not found. Downloading...")
    nltk.download('punkt')
    print("'punkt' tokenizer downloaded successfully.")

# 2. Load Penn Treebank tagged sentences
print("\nLoading Penn Treebank tagged sentences...")
corpus = treebank.tagged_sents()
print(f"Total sentences in corpus: {len(corpus)}")

print("\nExample of a tagged sentence:")
print(corpus[0])

# 3. Build vocabulary from the corpus
vocab = sorted({word for sent in corpus for (word, tag) in sent})

print(f"\nVocabulary size: {len(vocab)}")

print("\nFirst 50 words in vocabulary:")
print(vocab[:50])

print("\nLast 50 words in vocabulary:")
print(vocab[-50:])


Checking for 'punkt' tokenizer...
'punkt' tokenizer found.

Loading Penn Treebank tagged sentences...
Total sentences in corpus: 3914

Example of a tagged sentence:
[('Pierre', 'NNP'), ('Vinken', 'NNP'), (',', ','), ('61', 'CD'), ('years', 'NNS'), ('old', 'JJ'), (',', ','), ('will', 'MD'), ('join', 'VB'), ('the', 'DT'), ('board', 'NN'), ('as', 'IN'), ('a', 'DT'), ('nonexecutive', 'JJ'), ('director', 'NN'), ('Nov.', 'NNP'), ('29', 'CD'), ('.', '.')]

Vocabulary size: 12408

First 50 words in vocabulary:
['!', '#', '$', '%', '&', "'", "''", "'30s", "'40s", "'50s", "'80s", "'82", "'86", "'S", "'d", "'ll", "'m", "'re", "'s", "'ve", '*', '*-1', '*-10', '*-100', '*-101', '*-102', '*-103', '*-104', '*-105', '*-106', '*-107', '*-108', '*-109', '*-11', '*-110', '*-111', '*-112', '*-113', '*-114', '*-115', '*-116', '*-117', '*-118', '*-119', '*-12', '*-120', '*-121', '*-122', '*-123', '*-124']

Last 50 words in vocabulary:
['wrath', 'wrecking', 'wrenching', 'wrestling', 'wrists', 'write', 'write

In [4]:
# Build vocabulary from Treebank corpus
# A sorted list of unique words
vocab_list = sorted({word for sent in corpus for (word, tag) in sent})

# Create vocab dictionary: word → unique index
vocab = {word: idx for idx, word in enumerate(vocab_list)}

print("Vocabulary dictionary: word → unique integer ID")

# Print first 20 entries
cnt = 0
for word, idx in vocab.items():
    print(f"{word}: {idx}")
    cnt += 1
    if cnt > 20:
        break


Vocabulary dictionary: word → unique integer ID
!: 0
#: 1
$: 2
%: 3
&: 4
': 5
'': 6
'30s: 7
'40s: 8
'50s: 9
'80s: 10
'82: 11
'86: 12
'S: 13
'd: 14
'll: 15
'm: 16
're: 17
's: 18
've: 19
*: 20


In [5]:
import nltk
from nltk.corpus import treebank
import string

# Load the Penn Treebank test corpus
sentences = treebank.tagged_sents()

# Use last 10% as test corpus
split_idx = int(len(sentences) * 0.9)
test_corpus = sentences[split_idx:]   # list of [(word, tag), ...]

print("A sample of the test corpus:")
print(test_corpus[0:3])

A sample of the test corpus:
[[('Kalamazoo', 'NNP'), (',', ','), ('Mich.-based', 'JJ'), ('First', 'NNP'), ('of', 'IN'), ('America', 'NNP'), ('said', 'VBD'), ('0', '-NONE-'), ('it', 'PRP'), ('will', 'MD'), ('eliminate', 'VB'), ('the', 'DT'), ('13', 'CD'), ('management', 'NN'), ('positions', 'NNS'), ('of', 'IN'), ('the', 'DT'), ('former', 'JJ'), ('Midwest', 'NNP'), ('Financial', 'NNP'), ('parent', 'NN'), ('company', 'NN'), ('.', '.')], [('First', 'NNP'), ('of', 'IN'), ('America', 'NNP'), ('said', 'VBD'), ('0', '-NONE-'), ('some', 'DT'), ('of', 'IN'), ('the', 'DT'), ('managers', 'NNS'), ('will', 'MD'), ('take', 'VB'), ('other', 'JJ'), ('jobs', 'NNS'), ('with', 'IN'), ('First', 'NNP'), ('of', 'IN'), ('America', 'NNP'), ('.', '.')], [('But', 'CC'), ('it', 'PRP'), ('said', 'VBD'), ('that', 'IN'), ('severance', 'NN'), ('payments', 'NNS'), ('to', 'TO'), ('those', 'DT'), ('executives', 'NNS'), ('not', 'RB'), ('staying', 'VBG'), ('with', 'IN'), ('the', 'DT'), ('company', 'NN'), ('will', 'MD'), (

In [6]:
# Punctuation characters for unknown word handling
punct = set(string.punctuation)

# Morphology suffix rules
noun_suffix = ["action", "age", "ance", "cy", "dom", "ee", "ence", "er",
               "hood", "ion", "ism", "ist", "ity", "ling", "ment", "ness",
               "or", "ry", "scape", "ship", "ty"]
verb_suffix = ["ate", "ify", "ise", "ize"]
adj_suffix = ["able", "ese", "ful", "i", "ian", "ible", "ic", "ish", "ive", "less", "ly", "ous"]
adv_suffix = ["ward", "wards", "wise"]

# Unknown word assignment function
def assign_unk(tok):
    """
    Assign unknown word tokens using morphological and lexical rules.
    """
    # Digit-containing word
    if any(char.isdigit() for char in tok):
        return "--unk_digit--"

    # Punctuation tokens
    elif any(char in punct for char in tok):
        return "--unk_punct--"

    # Capitalized or upper-case word
    elif any(char.isupper() for char in tok):
        return "--unk_upper--"

    # Likely nouns based on suffix
    elif any(tok.endswith(suffix) for suffix in noun_suffix):
        return "--unk_noun--"

    # Likely verbs based on suffix
    elif any(tok.endswith(suffix) for suffix in verb_suffix):
        return "--unk_verb--"

    # Likely adjectives based on suffix
    elif any(tok.endswith(suffix) for suffix in adj_suffix):
        return "--unk_adj--"

    # Likely adverbs based on suffix
    elif any(tok.endswith(suffix) for suffix in adv_suffix):
        return "--unk_adv--"

    # Default unknown token
    return "--unk--"


In [7]:
def preprocess(vocab, sentences):
    """
    Preprocess Treebank sentences:
    - Flatten words
    - Handle unknown words
    - Replace sentence breaks with --n--
    """
    orig = []
    prep = []

    for sent in sentences:
        for word, tag in sent:

            # Save original
            orig.append(word)

            # Unknown word
            if word not in vocab:
                prep.append(assign_unk(word))
            else:
                prep.append(word)

        # Add sentence boundary marker
        orig.append("")
        prep.append("--n--")

    return orig, prep


In [8]:
train_sentences = sentences[:split_idx]
test_sentences  = sentences[split_idx:]

In [9]:
# corpus without tags, preprocessed
orig_test, prep = preprocess(vocab, test_sentences)

print('The length of the preprocessed test corpus:', len(prep))
print('This is a sample of the preprocessed test corpus:')
print(prep[0:10])


The length of the preprocessed test corpus: 10217
This is a sample of the preprocessed test corpus:
['Kalamazoo', ',', 'Mich.-based', 'First', 'of', 'America', 'said', '0', 'it', 'will']


---

# Section 2: Creating Dictionaries and Baseline POS Prediction

---

After preprocessing, the next step is to prepare the information the model needs to learn how words are linked to POS tags and how tags follow each other in sentences. We do this by creating three types of counts from the training data:

1. **Emission Counts**: Count how often a word appears with a particular tag. This tells the model the probability of a tag given a word.

2. **Transition Counts**: Count how often one tag follows another. This helps the model understand the natural order of tags in sentences.

3. **Tag Counts**: Count how often each tag appears in total. This is used to normalize probabilities and understand the overall tag distribution.

We loop through the training sentences and update these counts for every word-tag pair. Unknown words are replaced by special tokens based on their type, like numbers, punctuation, capital letters, or common suffixes. We also mark the start and end of sentences so the model knows sentence boundaries.

---

## Baseline POS Prediction

With these counts ready, we build a simple baseline tagger. For each word, we choose the tag that occurs most often with it in the training data. We then compare these predictions with the true tags in the test data. Words are counted as correct if the predicted tag matches the true tag. Empty lines or invalid word-tag pairs are skipped. The accuracy is calculated as the percentage of correct predictions.

This baseline gives a starting point by only looking at word-tag relationships, without considering tag sequences, and sets the stage for more advanced methods like HMM and Viterbi.


In [10]:
def get_word_tag(word, tag, vocab):
    """
    Handle unknown words and return (word, tag)
    """
    if word not in vocab:
        word = assign_unk(word)

    return word, tag


In [11]:
from collections import defaultdict

def create_dictionaries(training_sentences, vocab, verbose=True):
    """
    Input:
        training_sentences: list of sentences from Treebank
        vocab: dictionary of known words
    Output:
        emission_counts
        transition_counts
        tag_counts
    """

    emission_counts = defaultdict(int)
    transition_counts = defaultdict(int)
    tag_counts = defaultdict(int)

    # For transition model: start state
    prev_tag = "--s--"

    word_counter = 0

    # Loop through each sentence
    for sent in training_sentences:
        for word, tag in sent:
            word_counter += 1

            if verbose and word_counter % 50000 == 0:
                print(f"word count = {word_counter}")

            # Handle unknown words
            word, tag = get_word_tag(word, tag, vocab)

            # TRANSITION: previous tag → current tag
            transition_counts[(prev_tag, tag)] += 1

            # EMISSION: tag emits word
            emission_counts[(tag, word)] += 1

            # Count tag frequency
            tag_counts[tag] += 1

            # Update prev tag
            prev_tag = tag

        # End of sentence
        transition_counts[(prev_tag, "--s--")] += 1
        prev_tag = "--s--"

    return emission_counts, transition_counts, tag_counts


In [12]:
emission_counts, transition_counts, tag_counts = create_dictionaries(train_sentences, vocab)


word count = 50000


In [13]:
# get all the POS states
states = sorted(tag_counts.keys())
print(f"Number of POS tags (number of 'states'): {len(states)}")
print("View these POS tags (states)")
print(states)

Number of POS tags (number of 'states'): 46
View these POS tags (states)
['#', '$', "''", ',', '-LRB-', '-NONE-', '-RRB-', '.', ':', 'CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ', 'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNP', 'NNPS', 'NNS', 'PDT', 'POS', 'PRP', 'PRP$', 'RB', 'RBR', 'RBS', 'RP', 'SYM', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP', 'VBZ', 'WDT', 'WP', 'WP$', 'WRB', '``']


The 'states' are the Parts-of-speech designations found in the training data. They will also be referred to as 'tags' or POS in this Notebook. 

- "NN" is noun, singular, 
- 'NNS' is noun, plural. 
- In addition, there are helpful tags like '--s--' which indicate a start of a sentence.


In [14]:
print("transition examples:")
for ex in list(transition_counts.items())[:5]:
    print(ex)


transition examples:
(('--s--', 'NNP'), 681)
(('NNP', 'NNP'), 3264)
(('NNP', ','), 1335)
((',', 'CD'), 108)
(('CD', 'NNS'), 462)


In [15]:
print("\nemission examples:")
for ex in list(emission_counts.items())[200:203]:
    print(ex)



emission examples:
(('CD', '9.8'), 2)
(('CD', 'billion'), 125)
(('VBN', 'sold'), 23)


In [16]:
print("\nambiguous word example:")
for tup, cnt in emission_counts.items():
    if tup[1] == 'back':
        print(tup, cnt)



ambiguous word example:
('NN', 'back') 2
('VB', 'back') 1
('RB', 'back') 15
('RP', 'back') 3
('JJ', 'back') 1



### 1.2 - Testing

Now we will test the accuracy of our parts-of-speech tagger using your `emission_counts` dictionary. 


In [17]:
# Convert test sentences into "word POS" lines like WSJ_24.pos format
y = []
for sent in test_sentences:
    for word, tag in sent:
        y.append(f"{word} {tag}")
    y.append("")   # sentence boundary


In [18]:
def predict_pos(prep, y, emission_counts, vocab, states):
    """
    Baseline POS tag predictor:
    For each word, choose the POS tag with highest emission count.
    """

    num_correct = 0
    total = 0

    for word, y_line in zip(prep, y):

        # Skip empty sentence boundary
        if y_line.strip() == "":
            continue

        parts = y_line.split()
        if len(parts) != 2:
            continue

        true_word, true_tag = parts

        best_tag = None
        best_count = 0

        # Word is already preprocessed
        # If unknown → already replaced in preprocess()
        for tag in states:

            key = (tag, word)

            if key in emission_counts:
                count = emission_counts[key]

                if count > best_count:
                    best_count = count
                    best_tag = tag

        # Compare prediction with true label
        if best_tag == true_tag:
            num_correct += 1

        total += 1

    return num_correct / total


In [19]:
accuracy = predict_pos(prep, y, emission_counts, vocab, states)
print(f"Baseline prediction accuracy: {accuracy * 100:.2f}%")


Baseline prediction accuracy: 86.49%


---

# Section 3: Creating Transition and Emission Matrices

---

After counting how often words appear with tags and how tags follow each other, the next step is to convert these counts into **probabilities**. These probabilities are what a **Hidden Markov Model (HMM)** uses to predict sequences of tags for sentences. We do this by creating two main matrices:

1. **Transition Matrix (A)**:  
   This matrix shows how likely it is for one tag to follow another. Each value `A[i, j]` represents the probability of tag `j` coming after tag `i`. These probabilities are calculated by dividing the number of times a tag pair occurs by the total occurrences of the first tag. To handle cases where some tag pairs do not appear in the training data, we add a small value to all entries, a technique called **smoothing**, which prevents zero probabilities.

2. **Emission Matrix (B)**:  
   This matrix tells how likely a word is to appear for a particular tag. Each entry `B[i, j]` is the probability that the `j`-th word is generated by the `i`-th tag. We calculate this by dividing the count of a word-tag pair by the total count of that tag. We also apply smoothing here to handle cases where some words may not have appeared with certain tags in the training data.

---

## Role of These Matrices in HMM

* The **transition matrix** captures the structure of language at the tag level by modeling which tags are likely to follow others.  
* The **emission matrix** connects tags to the words they usually generate.  
* Together, these matrices form the core of a **Hidden Markov Model (HMM)**. In an HMM, the tags are **hidden states** that generate observable words. The transition and emission probabilities allow the model to compute the most likely sequence of tags for a sentence.

Using these matrices, we can apply algorithms like **Viterbi** to find the sequence of tags that has the highest probability, taking into account both the likelihood of each word given a tag and the likelihood of each tag sequence. Even without Viterbi, these matrices can give a rough estimate of the HMM’s performance by assigning each word the most probable tag according to emission probabilities alone, though this ignores the sequence information captured by the transitions.


In [20]:
import numpy as np

def create_transition_matrix(alpha, tag_counts, transition_counts):
    """
    Create the transition probability matrix A[i,j] = P(tag_j | tag_i)
    """
    states = sorted(tag_counts.keys())
    num_tags = len(states)
    A = np.zeros((num_tags, num_tags))

    for i in range(num_tags):
        prev_tag = states[i]
        prev_count = tag_counts[prev_tag]

        for j in range(num_tags):
            curr_tag = states[j]

            count = transition_counts.get((prev_tag, curr_tag), 0)

            # Add smoothing
            A[i, j] = (count + alpha) / (prev_count + alpha * num_tags)

    return A


In [21]:
alpha = 0.001
A = create_transition_matrix(alpha, tag_counts, transition_counts)

print("Transition matrix A sample:")
A_sub = pd.DataFrame(
    A[10:15, 10:15], 
    index=states[10:15], 
    columns=states[10:15]
)
print(A_sub)


Transition matrix A sample:
          CD        DT            EX            FW        IN
CD  0.178739  0.001582  3.163510e-07  3.163510e-07  0.037013
DT  0.023385  0.001360  1.359611e-07  1.360970e-04  0.010197
EX  0.000012  0.000012  1.175834e-05  1.175834e-05  0.000012
FW  0.000247  0.000247  2.471577e-04  2.471577e-04  0.000247
IN  0.063624  0.317557  1.124207e-03  1.125219e-04  0.016974


In [22]:
def create_emission_matrix(alpha, tag_counts, emission_counts, vocab):
    """
    Create emission matrix B[tag_i, word_j] = P(word_j | tag_i)
    vocab must be a LIST of words (not dict)
    """
    states = sorted(tag_counts.keys())
    num_tags = len(states)

    # Convert vocab dict → list of words in sorted order
    word_list = sorted(vocab.keys())
    num_words = len(word_list)

    B = np.zeros((num_tags, num_words))

    for i in range(num_tags):
        tag = states[i]
        tag_count = tag_counts[tag]

        for j in range(num_words):
            word = word_list[j]

            count = emission_counts.get((tag, word), 0)

            B[i, j] = (count + alpha) / (tag_count + alpha * num_words)

    return B


In [24]:
print("Emission matrix B sample:")

# 1. Create the emission matrix B
alpha = 0.001
B = create_emission_matrix(alpha, tag_counts, emission_counts, vocab)

# 2. Sample words
sample_words = ["725", "adroitly", "engineers", "promoted", "synergy"]

# Filter ONLY words that exist in vocab
valid_words = [w for w in sample_words if w in vocab]

# Map valid words → their vocab indices
cols = [vocab[w] for w in valid_words]

# Sample tags
sample_tags = ["CD", "NN", "NNS", "VB", "RB", "RP"]
rows = [states.index(tag) for tag in sample_tags]

# 3. Build submatrix
B_sub = pd.DataFrame(
    B[np.ix_(rows, cols)],
    index=sample_tags,
    columns=valid_words
)

print(B_sub)


Emission matrix B sample:
        engineers
CD   3.151186e-07
NN   8.521947e-08
NNS  3.659200e-04
VB   4.305876e-07
RB   3.852959e-07
RP   4.844773e-06


---

# Section 4: Viterbi Algorithm – Initialization

---

In part-of-speech (POS) tagging using a Hidden Markov Model (HMM), the **Viterbi algorithm** is used to find the most likely sequence of tags for a sentence. Before calculating probabilities for the whole sentence, the algorithm needs an **initialization step** to set up the necessary structures.

During initialization, we create two important matrices:

1. **Probability Matrix (`best_probs`)**:  
   This stores the highest probability of each tag at every position in the sentence. For the first word, the probability is calculated using the likelihood of starting with that tag (transition from a special start state) and the likelihood of that tag generating the observed word (emission probability).

2. **Backpointer Matrix (`best_paths`)**:  
   This keeps track of which previous tag gave the highest probability for the current tag. It is later used to reconstruct the most likely sequence of tags.

The first word is special because it has no previous tag. To handle this, we use a **start state** that represents the beginning of a sentence. The probabilities from the start state to each possible tag are combined with the emission probabilities for the first word. If the first word is not in the training vocabulary, we replace it with a special **unknown word token**. This allows the model to handle words it has never seen before.

Initialization is important because it sets a strong starting point for the forward pass. By correctly calculating probabilities for the first word and handling unknown words, the Viterbi algorithm can efficiently compute the most likely sequence of tags for the rest of the sentence.

In short, initialization prepares the Viterbi algorithm to predict POS tags by combining the structure of language (transition probabilities) and word-tag information (emission probabilities), while ensuring the model can handle words not seen during training.


In [25]:
# Add unknown tokens to vocab and emission matrix if not already
unk_tokens = ["--unk--", "--unk_digit--", "--unk_upper--", "--unk_punct--",
              "--unk_noun--", "--unk_verb--", "--unk_adj--", "--unk_adv--", "--n--"]

for token in unk_tokens:
    if token not in vocab:
        idx = len(vocab)
        vocab[token] = idx
        # Expand B to add a new column for this token
        B = np.hstack([B, np.full((B.shape[0], 1), 1e-8)])


In [26]:
import numpy as np

def initialize(states, tag_counts, A, B, corpus, vocab):
    """
    Viterbi initialization and forward pass.

    Input:
        states: list of POS tags
        tag_counts: dict mapping tag -> count
        A: Transition matrix (num_tags x num_tags)
        B: Emission matrix (num_tags x vocab_size)
        corpus: list of words to tag (preprocessed)
        vocab: dict mapping word -> index

    Output:
        best_probs: np.array (num_tags x len(corpus)) of probabilities
        best_paths: np.array (num_tags x len(corpus)) of backpointers
    """
    
    num_tags = len(states)
    seq_len = len(corpus)
    
    best_probs = np.zeros((num_tags, seq_len))
    best_paths = np.zeros((num_tags, seq_len), dtype=int)
    
    # First word initialization
    word = corpus[0]
    if word not in vocab:
        word = assign_unk(word)
    
    for i, tag in enumerate(states):
        # Assuming start state is "--s--"
        start_idx = states.index("--s--") if "--s--" in states else None
        transition_prob = A[start_idx, i] if start_idx is not None else 1.0 / num_tags
        emission_prob = B[i, vocab[word]] if word in vocab else 1e-8  # very small for unknowns
        best_probs[i, 0] = transition_prob * emission_prob
        best_paths[i, 0] = 0  # no previous tag at t=0
    
    # Forward pass for t = 1 to T-1
    for t in range(1, seq_len):
        word = corpus[t]
        if word not in vocab:
            word = assign_unk(word)
        
        for j, curr_tag in enumerate(states):
            max_prob = -1
            max_state = 0
            for i, prev_tag in enumerate(states):
                prob = best_probs[i, t-1] * A[i, j]
                if prob > max_prob:
                    max_prob = prob
                    max_state = i
            emission_prob = B[j, vocab[word]] if word in vocab else 1e-8
            best_probs[j, t] = max_prob * emission_prob
            best_paths[j, t] = max_state
            
    return best_probs, best_paths


In [27]:
best_probs, best_paths = initialize(states, tag_counts, A, B, prep, vocab)

print(f"best_probs[0,0]: {best_probs[0,0]:.4f}")
print(f"best_paths[2,3]: {best_paths[2,3]}")


best_probs[0,0]: 0.0000
best_paths[2,3]: 2


---

# Section 4.1: Viterbi Forward Pass

---

After initializing the model, the next step is the **Viterbi forward pass**, which helps us find the most likely sequence of POS tags for a sentence. In this step, we go through the sentence one word at a time and calculate probabilities for all possible tags.

For each word, the algorithm looks at every possible current tag and finds which previous tag gives the highest probability of reaching the current tag. It uses two kinds of probabilities:

* **Transition probability** – the chance of one tag coming after another tag.  
* **Emission probability** – the chance of a tag generating the current word.

The forward pass keeps track of two things:

* A **probability matrix** that stores the highest probability for each tag at each word.  
* A **backpointer matrix** that remembers which previous tag gave this highest probability.

By using these two matrices, the model can later find the best sequence of tags for the sentence. To avoid very small probabilities becoming zero, we often work with **log probabilities**, which makes calculations safer.

Special tokens are used for **unknown words** so that even words not seen in training can get a reasonable tag.

In simple words, the Viterbi forward pass combines information about the **structure of language** (how tags follow each other) and **word-tag information** (which words belong to which tags) to figure out the most likely tag for every word in the sentence. The result prepares us to find the final best sequence of tags using the **backward pass**.


In [28]:
import numpy as np

def viterbi_forward_log(A, B, corpus, vocab, states, verbose=False):
    """
    Viterbi forward algorithm using log probabilities to prevent underflow.
    """
    num_tags = len(states)
    seq_len = len(corpus)
    
    best_probs = np.full((num_tags, seq_len), -np.inf)  # log(0) = -inf
    best_paths = np.zeros((num_tags, seq_len), dtype=int)
    
    start_idx = states.index("--s--") if "--s--" in states else None
    
    # Initialization
    word = corpus[0]
    if word not in vocab:
        word = assign_unk(word)
    
    for i, tag in enumerate(states):
        trans_prob = A[start_idx, i] if start_idx is not None else 1.0 / num_tags
        emis_prob = B[i, vocab[word]] if word in vocab else 1e-8
        best_probs[i, 0] = np.log(trans_prob) + np.log(emis_prob)
        best_paths[i, 0] = 0
    
    # Forward recursion
    for t in range(1, seq_len):
        word = corpus[t]
        if word not in vocab:
            word = assign_unk(word)
        
        for j, curr_tag in enumerate(states):
            max_prob = -np.inf
            max_state = 0
            for i, prev_tag in enumerate(states):
                prob = best_probs[i, t-1] + np.log(A[i, j])
                if prob > max_prob:
                    max_prob = prob
                    max_state = i
            emis_prob = B[j, vocab[word]] if word in vocab else 1e-8
            best_probs[j, t] = max_prob + np.log(emis_prob)
            best_paths[j, t] = max_state
        
        if verbose and t % 100 == 0:
            print(f"Processed word {t}/{seq_len}: {corpus[t]}")
    
    return best_probs, best_paths


In [29]:
best_probs, best_paths = viterbi_forward_log(A, B, prep, vocab, states)

print(f"best_probs[0,1]: {best_probs[0,1]:.4f}")  # negative log-prob
print(f"best_probs[0,4]: {best_probs[0,4]:.4f}")


best_probs[0,1]: -30.5394
best_probs[0,4]: -65.3522


---

# Section 4.2: Viterbi Backward Pass

---

Once the forward pass has calculated the probabilities of all tags at each word, the **Viterbi backward pass** is used to find the actual sequence of tags for the sentence. This step is also called **backtracking**.

The backward pass starts from the last word in the sentence. It looks at the **probability matrix** from the forward pass and picks the tag with the highest probability at the final word. This tag becomes the last tag in the predicted sequence.

Then, using the **backpointer matrix** from the forward pass, the algorithm traces backwards through the sentence. For each word, it finds which previous tag led to the current tag with the highest probability. This way, it reconstructs the most likely sequence of tags from the last word back to the first word.

After reaching the first word, the sequence is reversed to restore the correct order. Finally, the **tag indices** are converted back into their actual POS tag names, giving the final predicted sequence of tags for the entire sentence.

In simple terms, the backward pass uses the information stored during the forward pass to assemble the **best possible tag sequence**. While the forward pass calculates probabilities, the backward pass actually tells us which tags to assign to each word. Together, the forward and backward passes make the Viterbi algorithm an effective method for POS tagging, combining the likelihood of **tag transitions** and **word-tag associations** to predict accurate sequences.


In [30]:
def viterbi_backward(best_probs, best_paths, states):
    """
    Backtracking function for Viterbi algorithm.

    Input:
        best_probs: np.array of log-probabilities (num_tags x sequence length)
        best_paths: np.array of backpointers (num_tags x sequence length)
        states: list of POS tags corresponding to rows of best_probs

    Output:
        tag_sequence: list of predicted POS tags
    """
    seq_len = best_probs.shape[1]

    # Start from the tag with maximum probability at last position
    last_tag_idx = np.argmax(best_probs[:, -1])
    best_sequence = [last_tag_idx]

    # Backtrack through the best_paths
    for t in range(seq_len - 1, 0, -1):
        last_tag_idx = best_paths[last_tag_idx, t]
        best_sequence.append(last_tag_idx)

    # Reverse the sequence to get correct order
    best_sequence = best_sequence[::-1]

    # Convert indices to tag names
    tag_sequence = [states[i] for i in best_sequence]

    return tag_sequence


In [31]:
pred = viterbi_backward(best_probs, best_paths, states)
m = len(pred)

print('The prediction for the last 7 words is:')
print('Words:', prep[-7:m-1])
print('Predicted tags:', pred[-7:m-1], "\n")

print('The prediction for the first 7 words is:')
print('Words:', prep[0:7])
print('Predicted tags:', pred[0:7])


The prediction for the last 7 words is:
Words: ['first', 'quarter', 'of', 'next', 'year', '.']
Predicted tags: ['JJ', 'NN', 'IN', 'JJ', 'NN', '.'] 

The prediction for the first 7 words is:
Words: ['Kalamazoo', ',', 'Mich.-based', 'First', 'of', 'America', 'said']
Predicted tags: ['UH', ',', 'CC', 'NNP', 'IN', 'NNP', 'VBD']


---

# Section 5: Predicting and Evaluating POS Tags

---

After running the **Viterbi algorithm**, we predict the POS tag for each word in the test set. These predictions are compared with the **true tags** to compute **accuracy**, which measures the fraction of correctly predicted tags. Sentence boundaries and invalid lines are ignored. 

This evaluation shows how well the **Hidden Markov Model (HMM)** with Viterbi handles both **known and unknown words**, providing a clear measure of the model’s performance on **unseen data**.


In [32]:
def compute_accuracy(pred, y):
    """
    Compute accuracy of predicted POS tags against ground truth.

    Input: 
        pred: list of predicted POS tags (from Viterbi)
        y: list of lines, each in "word tag" format (e.g., 'dog NN')
    Output: 
        accuracy: fraction of correct predictions
    """
    num_correct = 0
    total = 0
    
    for prediction, y_line in zip(pred, y):
        # Split the line into word and tag
        word_tag = y_line.split()
        
        # Skip lines that don't have exactly 2 items (e.g., sentence boundaries)
        if len(word_tag) != 2:
            continue
        
        word, true_tag = word_tag
        
        # Compare predicted tag with true tag
        if prediction == true_tag:
            num_correct += 1
        
        total += 1

    return num_correct / total


In [33]:
# pred is the list of POS tags predicted by viterbi_backward
# y is the list of "word tag" lines from your test corpus

accuracy = compute_accuracy(pred, y)
print(f"Accuracy of the Viterbi algorithm is {accuracy*100:.2f}%")


Accuracy of the Viterbi algorithm is 90.24%
